In [1]:
from sklearn import svm,datasets
iris = datasets.load_iris()

In [2]:
dir(iris)

['DESCR',
 'data',
 'data_module',
 'feature_names',
 'filename',
 'frame',
 'target',
 'target_names']

In [6]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [8]:
import pandas as pd 
dd = pd.DataFrame(iris.data, columns = iris.feature_names)
dd["flower"]=iris.target
dd["flower"] = dd["flower"].apply(lambda x :iris.target_names[x])
dd[47:55]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
47,4.6,3.2,1.4,0.2,"[setosa, versicolor, virginica]"
48,5.3,3.7,1.5,0.2,"[setosa, versicolor, virginica]"
49,5.0,3.3,1.4,0.2,"[setosa, versicolor, virginica]"
50,7.0,3.2,4.7,1.4,"[setosa, versicolor, virginica]"
51,6.4,3.2,4.5,1.5,"[setosa, versicolor, virginica]"
52,6.9,3.1,4.9,1.5,"[setosa, versicolor, virginica]"
53,5.5,2.3,4.0,1.3,"[setosa, versicolor, virginica]"
54,6.5,2.8,4.6,1.5,"[setosa, versicolor, virginica]"


In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test =train_test_split(iris.data,iris.target,test_size=0.3)

In [14]:
model = svm.SVC(kernel="rbf",C =30, gamma="auto")
model.fit(X_train,Y_train)
model.score(X_test,Y_test)

1.0

<h4>Manually try suppling models with different parameters to cross_val_score function with 5 fold cross validation<h4>

In [17]:
from sklearn.model_selection  import cross_val_score

In [19]:
cross_val_score(svm.SVC(kernel='linear',C=10,gamma='auto'),iris.data, iris.target, cv=5)

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [21]:
cross_val_score(svm.SVC(kernel='rbf',C=10,gamma='auto'),iris.data, iris.target, cv=5)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [22]:
cross_val_score(svm.SVC(kernel='linear',C=30,gamma='auto'),iris.data, iris.target, cv=5)

array([1. , 1. , 0.9, 0.9, 1. ])

<h2>Above approach is tiresome and very manual. We can use for loop as an alternative</h2>

In [24]:
import numpy as np

In [25]:
kernels = ['rbf', 'linear']
C = [1,10,20]
avg_scores = {}
for kval in kernels:
    for cval in C:
        cv_scores = cross_val_score(svm.SVC(kernel=kval,C=cval,gamma='auto'),iris.data, iris.target, cv=5)
        avg_scores[kval + '_' + str(cval)] = np.average(cv_scores)

avg_scores

{'rbf_1': 0.9800000000000001,
 'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9666666666666668,
 'linear_1': 0.9800000000000001,
 'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666}

<h2 style = color:blue>Approach 3: Use GridSearchCV</h3>

In [26]:
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(svm.SVC(gamma='auto'), {
    'C': [1,10,20],
    'kernel': ['rbf','linear']
}, cv=5, return_train_score=False)
clf.fit(iris.data, iris.target)
clf.cv_results_

{'mean_fit_time': array([0.00321002, 0.00333571, 0.00140176, 0.00216556, 0.00646725,
        0.00039563]),
 'std_fit_time': array([0.00642004, 0.00630045, 0.00280352, 0.00312721, 0.01293449,
        0.00079126]),
 'mean_score_time': array([0.00140605, 0.01497374, 0.00180621, 0.00019612, 0.00020862,
        0.00446038]),
 'std_score_time': array([0.0028121 , 0.02634992, 0.00361242, 0.00039225, 0.00041723,
        0.00619282]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20, 'kernel': 'linear'}],


In [28]:

df = pd.DataFrame(clf.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003210,0.006420,0.001406,0.002812,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.003336,0.006300,0.014974,0.026350,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.001402,0.002804,0.001806,0.003612,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.002166,0.003127,0.000196,0.000392,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
4,0.006467,0.012934,0.000209,0.000417,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,5
5,0.000396,0.000791,0.004460,0.006193,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,6


In [30]:
df[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [32]:
clf.best_params_

{'C': 1, 'kernel': 'rbf'}

In [34]:
clf.best_estimator_

SVC(C=1, gamma='auto')

In [35]:
clf.best_score_

0.9800000000000001

How about different models with different hyperparameters?


In [37]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    }
}

In [38]:
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(iris.data, iris.target)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.966667,{'n_estimators': 5}
2,logistic_regression,0.966667,{'C': 5}
